In [1]:
from typing import Annotated, Sequence, TypedDict, List, Dict
from dotenv import load_dotenv  
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END, START
from langchain_groq import ChatGroq
from langgraph.prebuilt import ToolNode
import os
import json
import requests
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import MSO_ANCHOR
from io import BytesIO


In [2]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

llm = ChatGroq(model="llama-3.1-8b-instant")

In [3]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    ppt_output_path: str
    images_path: List[str]
    audio_output_path: List[str]
    video_output_path: str

In [4]:
import json
from typing import List


def load_json_to_agent_state(json_path: str) -> AgentState:
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    return AgentState(
        messages=[],
        ppt_output_path=data.get("ppt_output_path", ""),
        audio_output_path=data.get("audio_output_path", [])
    )


In [ ]:

import tempfile
from pptx import Presentation
from moviepy import *



import os
from win32com.client import Dispatch


import os
from win32com.client import Dispatch

def export_ppt_to_images(ppt_path: str, export_folder: str, state: dict) -> None:
    """
    Opens a PowerPoint presentation, exports each slide as a PNG image,
    and saves them into the specified export folder.
    Also updates the state with paths to the exported images.
    """
    ppt_path = os.path.abspath(ppt_path)
    export_folder = os.path.abspath(export_folder)

    if not os.path.exists(ppt_path):
        raise FileNotFoundError(f"PPT file not found at {ppt_path}")

    if not os.path.exists(export_folder):
        os.makedirs(export_folder)

    powerpoint = Dispatch("PowerPoint.Application")
    powerpoint.Visible = 1

    try:
        presentation = powerpoint.Presentations.Open(ppt_path, WithWindow=False)
        presentation.SaveAs(export_folder, 17)  # 17 = PNG
        presentation.Close()
    finally:
        powerpoint.Quit()

    
    folder_path = "../assets/images"

    num_images = len([
        f for f in os.listdir(folder_path)
        if f.lower().endswith(".jpg")  # Ensure case-insensitive match
    ])

    print(f"Number of images: {num_images}")

    imageaddress = []
    for i in range(1, num_images + 1):
        imageaddress.append(f"../assets/images\\Slide{i}.jpg")

    state["images_path"] = imageaddress

    print(state["images_path"])


    print(f"Slides exported to: {export_folder}")

    import os

    import os
import win32com.client

def embed_audio_in_ppt(agent_state: dict) -> dict:
    ppt_path = os.path.abspath(agent_state["ppt_output_path"])
    audio_paths = agent_state["audio_output_path"]  # List of audio file paths (WAV)
    
    if not os.path.exists(ppt_path):
        raise FileNotFoundError(f"[ERROR] PPT file not found at: {ppt_path}")
    if not audio_paths or not isinstance(audio_paths, list):
        raise ValueError("[ERROR] 'audio_output_path' must be a list of audio file paths.")

    # Open PowerPoint application
    powerpoint = win32com.client.Dispatch("PowerPoint.Application")
    powerpoint.Visible = True

    # Open the presentation
    presentation = powerpoint.Presentations.Open(ppt_path, WithWindow=False)

    slide_count = presentation.Slides.Count
    if slide_count != len(audio_paths):
        print(f"[WARNING] Number of slides ({slide_count}) != number of audio files ({len(audio_paths)})")

    # Embed each audio into its respective slide
    for idx, audio_path in enumerate(audio_paths):
        abs_audio_path = os.path.abspath(audio_path)
        if not os.path.exists(abs_audio_path):
            print(f"[WARNING] Audio file not found: {abs_audio_path}")
            continue

        slide_index = idx + 1
        if slide_index > slide_count:
            break

        slide = presentation.Slides(slide_index)

        # Embed audio file into the slide
        shape = slide.Shapes.AddMediaObject2(
            FileName=abs_audio_path,
            LinkToFile=False,
            SaveWithDocument=True,
            Left=10,
            Top=10
        )

        shape.AnimationSettings.PlaySettings.PlayOnEntry = True
        shape.AnimationSettings.PlaySettings.HideWhileNotPlaying = True
        
        
        print(f"✅ Embedded audio to Slide {slide_index}")

    # Save the updated presentation
    new_ppt_path = ppt_path.replace(".pptx", "_with_audio.pptx")
    presentation.SaveAs(new_ppt_path)
    print(f"💾 Presentation saved as {new_ppt_path}")


    mp4_output_path = new_ppt_path.replace(".pptx", ".mp4")
    print(f"📽️ Exporting to video: {mp4_output_path}")
    presentation.CreateVideo(
        FileName=mp4_output_path,
        UseTimingsAndNarrations=True,
        VertResolution=720,
        FramesPerSecond=30,
        Quality=85
    )

    # Wait until video export is done
    while presentation.CreateVideoStatus != 3:  # 3 = ppMediaTaskStatusDone
        pass

    # Close PowerPoint
    presentation.Close()
    powerpoint.Quit()

    agent_state["video_output_path"] = mp4_output_path
    return agent_state


In [22]:
def Load_data_node(state: AgentState) -> AgentState:
    return(load_json_to_agent_state("../assets/scripts/paths.json"))




In [23]:
builder = StateGraph(AgentState)

builder.add_node("LoadData", Load_data_node)
builder.add_node("CreateVideo", embed_audio_in_ppt)

builder.add_edge(START, "LoadData")
builder.add_edge("LoadData", "CreateVideo")
builder.add_edge("CreateVideo", END)

graph = builder.compile()

In [24]:
initial_state = {"json_path": "../assets/scripts/paths.json"}
final_state = graph.invoke(initial_state)

print("Video Output Path:", final_state["video_output_path"])


[WARNING] Number of slides (21) != number of audio files (14)
✅ Embedded audio to Slide 1
✅ Embedded audio to Slide 2
✅ Embedded audio to Slide 3
✅ Embedded audio to Slide 4
✅ Embedded audio to Slide 5
✅ Embedded audio to Slide 6
✅ Embedded audio to Slide 7
✅ Embedded audio to Slide 8
✅ Embedded audio to Slide 9
✅ Embedded audio to Slide 10
✅ Embedded audio to Slide 11
✅ Embedded audio to Slide 12
✅ Embedded audio to Slide 13
✅ Embedded audio to Slide 14


com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft PowerPoint', 'Presentation.Save : This presentation is read-only and must be saved with a different name.', '', 0, -2147467259), None)

In [ ]:
print("PPT Output Path:", final_state["ppt_output_path"])
print("Audio Output Paths:", final_state["images_path"])